In [ ]:
# Importamos librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input, GRU
from sklearn.metrics import mean_squared_error
import math
from keras import callbacks
from pathlib import Path 

# En caso de que usemos Google Colab
from google.colab import drive
drive.mount('/content/drive')

# Cargamos el dataset
raw_data = pd.read_csv('/content/drive/MyDrive/Projecto Marvik/Datos/TWTR.csv')
# raw_data = pd.read_csv('/content/drive/MyDrive/Projecto Marvik/Datos/META.csv')
# raw_data = pd.read_csv('/content/drive/MyDrive/Projecto Marvik/Datos/S&P500.csv') # sacado de NASDAQ

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def select_model(raw_data, lag, gru_neurons, epochs, optimizer):

  # Nos quedamos solamente con el valor de cierre de la accion
  raw_data = raw_data[['Close']]

  # Normalizamos el dataset
  dataset = raw_data.values
  dataset = dataset.astype('float32')
  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset)

  # Funcion para crear el dataset que le vamos a pasar al modelo, teniendo en cuenta el lag
  def create_dataset(dataset, lag):
      dataX, dataY = np.zeros((len(dataset)-lag,lag)), np.zeros(len(dataset)-lag)
      for i in range(len(dataset)-lag):
          dataX[i,:] = dataset[i:i+lag,0]
          dataY[i] = dataset[i+lag,0]
      return dataX, dataY

  # Ejecutamos la funcion de create_dataset y establecemos el lag
  datasetX, datasetY = create_dataset(dataset, lag)

  # Separamos el dataset en train y test
  train_size = int(len(datasetX) * 0.67)
  test_size = len(datasetX) - train_size
  trainX, testX = datasetX[0:train_size,:], datasetX[train_size:len(dataset),:]
  trainY, testY = datasetY[0:train_size], datasetY[train_size:len(dataset)]

  # Agregamos una dimensión para poder pasarle luego al modelo
  trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
  testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

  # Capas y armado del modelo
  model = Sequential([GRU(gru_neurons, input_shape=(1, trainX.shape[2])),
                      Dense(gru_neurons/2, activation='relu'),
                      Dense(gru_neurons/2, activation='relu'),
                      Dense(1)])

  # Seleccion de metricas y entrenamiento del modelo
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics= "MeanSquaredError")
  
  # Recupera los mejores weights que el modelo encontro en todas las corridas
  earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
  
  model.fit(trainX, trainY, batch_size=1, epochs=epochs, verbose=1, validation_split=0.1, callbacks=[earlystopping])

  score = model.evaluate(testX, testY, verbose = 1)

  # Hacemos las predicciones
  trainPredict = model.predict(trainX)
  testPredict = model.predict(testX)

  # Invertimos la normalizacion de los datos para poder calcular el RMSE
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])

  # Usamos el RMSE como métrica para evaluar la eficiencia del modelo
  # Train Score
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
  # Test Score
  testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))

  # Creamos nuevos datasets para graficar las predicciones
  # Train
  trainPredictPlot = np.empty_like(dataset)
  trainPredictPlot[:, :] = np.nan
  trainPredictPlot[lag:len(trainPredict)+lag, :] = trainPredict

  # Test
  testPredictPlot = np.empty_like(dataset)
  testPredictPlot[:, :] = np.nan
  testPredictPlot[len(trainPredict)+lag:, :] = testPredict

  return (trainScore, testScore, trainPredictPlot, testPredictPlot, model, scaler, optimizer, lag)

In [ ]:
# Prueba con diferentes optimizers para hallar el optimo en base al RMSE
raw_data = raw_data
gru_neurons = 64
epochs = 50
lags = range(1,11)
# optimizers = ['Adadelta','Adagrad','Adam','Adamax','Ftrl','Nadam','RMSprop','SGD']
optimizers = ['Adam','Adamax','Nadam','RMSprop']

RMSE_opt = [select_model(raw_data, lag, gru_neurons, epochs, optim) for optim in optimizers for lag in lags]

In [ ]:
opt, lag_model, RMSE_test = [] , [] , []
for i, j, k, l, m, n, o, p in RMSE_opt:
  RMSE_test.append(j)
  opt.append(o)
  lag_model.append(p)

results = pd.concat([ pd.Series(opt), pd.Series(lag_model), pd.Series(RMSE_test)], axis=1)
results.rename(columns={0:'opt', 1:'lags', 2:'RMSE_test'}, inplace=True)
results = pd.pivot(results, index='opt', values='RMSE_test', columns='lags')
print(results)

# Mejor RMSE obtenido en todos los modelos
print('Mejor optimizer: ' + str(results.index[np.argmin(results.min(axis=1))]))
print('Mejor numero de lags: ' + str(results.T.index[np.argmin(results.min(axis=0))]))

In [ ]:
# Guardamos tabla de resultados
filepath = Path('/content/drive/MyDrive/Projecto Marvik/results.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
results.to_csv(filepath , sep=',' ) 

In [ ]:
results.T.plot(legend="best_fit", figsize=(12,5), xticks=range(1,len(lags)+1), \
               title='RMSE plot', marker='o', ylabel='RMSE')

In [ ]:
# Generamos una funcion para utilizar todo el dataset en el entrenamiento
def run_model(raw_data, lag, gru_neurons, epochs, optimizer):

  # Nos quedamos solamente con el valor de cierre de la accion
  raw_data = raw_data[['Close']]

  # Normalizamos el dataset
  dataset = raw_data.values
  dataset = dataset.astype('float32')
  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset)

  # Funcion para crear el dataset que le vamos a pasar al modelo, teniendo en cuenta el lag
  def create_dataset(dataset, lag):
      dataX, dataY = np.zeros((len(dataset)-lag,lag)), np.zeros(len(dataset)-lag)
      for i in range(len(dataset)-lag):
          dataX[i,:] = dataset[i:i+lag,0]
          dataY[i] = dataset[i+lag,0]
      return dataX, dataY

  # Ejecutamos la funcion de create_dataset y establecemos el lag
  datasetX, datasetY = create_dataset(dataset, lag)

  # Separamos el dataset en train y test
  train_size = int(len(datasetX))
  
  trainX = datasetX[0:train_size,:]
  trainY = datasetY[0:train_size]

  # Agregamos una dimensión para poder pasarle luego al modelo
  trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

  # Capas y armado del modelo
  model = Sequential([GRU(gru_neurons, input_shape=(1, trainX.shape[2])),
                      Dense(gru_neurons/2, activation='relu'),
                      Dense(gru_neurons/2, activation='relu'),
                      Dense(1)])

  # Seleccion de metricas y entrenamiento del modelo
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics= "MeanSquaredError")
  
  # Recupera los mejores weights que el modelo encontro en todas las corridas
  earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
  
  model.fit(trainX, trainY, batch_size=1, epochs=epochs, verbose=1, validation_split=0.1, callbacks=[earlystopping])

  # Hacemos las predicciones
  trainPredict = model.predict(trainX)

  # Invertimos la normalizacion de los datos para poder calcular el RMSE
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])

  # Usamos el RMSE como métrica para evaluar la eficiencia del modelo
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))

  # Creamos nuevos datasets para graficar las predicciones
  trainPredictPlot = np.empty_like(dataset)
  trainPredictPlot[:, :] = np.nan
  trainPredictPlot[lag:len(trainPredict)+lag, :] = trainPredict

  return (trainScore, trainPredictPlot, model, scaler)

In [ ]:
# Luego de optimizar los parametros, entrenamos el modelos con todo el dataset
raw_data = raw_data
gru_neurons = 64
epochs = 50
best_optimizer = results.index[np.argmin(results.min(axis=1))]
best_lag = results.T.index[np.argmin(results.min(axis=0))]

# Corremos la funcion
trainScore, trainPredictPlot, model, scaler = run_model(raw_data, best_lag, gru_neurons, epochs, best_optimizer)

In [ ]:
# Graficamos los resultados del nuevo modelo
plt.figure(figsize=(25,7))
plt.plot(raw_data['Close'])
plt.plot(trainPredictPlot[:])
plt.legend(['Original Time Series','trainPredictPlot'])
plt.show()

In [ ]:
model.summary()

In [ ]:
def predict_values(days, trainPredict, scaler): 
  future_preds = trainPredictPlot[-best_lag:]

  for i in range(1, days+1):
    preds_scaled = scaler.transform(future_preds[-best_lag:,:])
    pred_val = scaler.inverse_transform(model.predict(preds_scaled.reshape(1,-1,best_lag)))
    future_preds = np.append(future_preds, pred_val, axis=0)

  future_preds = future_preds[best_lag-1:]
  return future_preds

In [ ]:
future_preds = predict_values(10, trainPredictPlot, scaler)

In [ ]:
future_preds_plot = np.empty((len(trainPredictPlot)+len(future_preds), 1))
future_preds_plot[:, :] = np.nan
future_preds_plot[len(trainPredictPlot):, :] = future_preds

In [ ]:
# Graficamos los ultimos 100 dias del dataset mas la prediccion
plt.figure(figsize=(20,7))
plt.plot(trainPredictPlot[-100:])
plt.plot(future_preds_plot[-100-len(future_preds)+1:])
plt.legend(['trainPredictPlot','future_preds_plot'])
plt.show()